In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import math
from scipy import stats
import xlsxwriter
from statistics import mean

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def stock_data_to_csv(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content, 'html.parser')
    
    raw_stock_prices = []
    
    s = soup.select('td.right')
    for element in s:
        raw_stock_prices.append(element.text)
    
    stock_name = soup.select('td.left')
    
    values =[]
    ticker =[]
    for i in range(0, len(raw_stock_prices)):
        price = raw_stock_prices[i]
        j = i+1
        if j%5 != 0:
            values.append(price.split()[1])
        else:
            values.append(price.split()[0])
        
    for element in stock_name:
        ticker.append(element.text)
    
    
    stock_float = []
    processed_data = []
    for i in values:
        if i.strip() == '-':
            processed_data.append('0')
        else:
            processed_data.append(i.strip())
    
    for i in processed_data:
        integer = float(i)
        stock_float.append(integer)
    
    rows1 = []
    current_row1 = []       
        
    for entry in stock_float:
        current_row1.append(entry)
        if len(current_row1) == 5:
            rows1.append(current_row1)
            current_row1 = []
    if current_row1:
        rows1.append(current_row1)
        # print(rows1)
    matrix = np.array(rows1)
    
    
    hqm_columns = [
                'Stock Name', 
                '1 Year Return', 
                '1 Year Return Percentile',
                '9 Month Return',
                '9 Month Return Percentile',
                '6 Month Return',
                '6 Month Return Percentile',
                '3 Month Return',
                '3 Month Return Percentile',
                'Price',
                'HQM Score',
                'No. of shares to buy'
                ]
    stock_hqm = pd.DataFrame(columns = hqm_columns)
    for symbol in ticker:
        stock_hqm = stock_hqm.append(
            pd.Series([
                symbol,
                'N/A',
                'N/A',
                'N/A',
                'N/A',
                'N/A',
                'N/A',
                'N/A',
                'N/A',
                'N/A',
                'N/A',
                'N/A'
            ], index = hqm_columns), ignore_index = True)
    for i in range(len(matrix)):
        stock_hqm.loc[i,'1 Year Return'] = matrix[i][0]
    for i in range(len(matrix)):
        stock_hqm.loc[i,'9 Month Return'] = matrix[i][1]
    for i in range(len(matrix)):
        stock_hqm.loc[i,'6 Month Return'] = matrix[i][2]
    for i in range(len(matrix)):
        stock_hqm.loc[i,'3 Month Return'] = matrix[i][3]
    for i in range(len(matrix)):
        stock_hqm.loc[i,'Price'] = matrix[i][4]
        
    
    time_periods= [
            '1 Year',
            '9 Month',
            '6 Month',
            '3 Month'
            ]
    for row in stock_hqm.index:
        for time in time_periods:
            change_col = f'{time} Return'
            percentile_col = f'{time} Return Percentile'
            stock_hqm.loc[row, percentile_col] = stats.percentileofscore(stock_hqm[change_col], stock_hqm.loc[row, change_col])
    
    for row in stock_hqm.index:
        momentum_percentiles = []
        for time in time_periods:
            momentum_percentiles.append(stock_hqm.loc[row, f'{time} Return Percentile'])
        stock_hqm.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    stock_hqm.sort_values('HQM Score', ascending = False, inplace = True)
    stock_hqm.reset_index(inplace = True, drop = True)
    stock_hqm
#     portfolio_input()
    
    position_size = float(10000000)/len(stock_hqm.index)
    for i in stock_hqm.index:
        stock_hqm.loc[i, "No. of shares to buy"] = math.floor(position_size/stock_hqm.loc[i, "Price"]) 
    print('Completed')
    return stock_hqm
    

In [5]:
url = 'https://www.moneycontrol.com/stocks/sectors/'
data_sector = requests.get(url)

dsoup1 = BeautifulSoup(data_sector.content, 'html.parser')
sector = []
s = dsoup1.select('ul.sec_comp_nm')
for element in s:
    sector.append(element)
sector_data = sector[2]
links = []
for i in sector_data:
    links.append(i)
data = links[1:-1]

sector_name = []
for i in data:
    sector_name.append(i.text)


In [6]:
list = []
for i in data:
    list.append(str(i))
sector_links = []
for i in list:
    start_index = i.find('href="') + len('href="')
    end_index = i.find('"', start_index)

    link = i[start_index:end_index]

    sector_links.append(link)
stock_sectors = dict(zip(sector_name, sector_links))

In [7]:
for i in  stock_sectors.values():
    print(i)

http://www.moneycontrol.com/stocks/sectors/abrasives.html
http://www.moneycontrol.com/stocks/sectors/aluminium.html
http://www.moneycontrol.com/stocks/sectors/aquaculture.html
http://www.moneycontrol.com/stocks/sectors/auto-2-3-wheelers.html
http://www.moneycontrol.com/stocks/sectors/auto-cars-jeeps.html
http://www.moneycontrol.com/stocks/sectors/auto-lcvshcvs.html
http://www.moneycontrol.com/stocks/sectors/auto-tractors.html
http://www.moneycontrol.com/stocks/sectors/auto-ancillaries.html
http://www.moneycontrol.com/stocks/sectors/banks-private-sector.html
http://www.moneycontrol.com/stocks/sectors/banks-public-sector.html
http://www.moneycontrol.com/stocks/sectors/bearings.html
http://www.moneycontrol.com/stocks/sectors/breweries-distilleries.html
http://www.moneycontrol.com/stocks/sectors/cables-powerothers.html
http://www.moneycontrol.com/stocks/sectors/cables-telephone.html
http://www.moneycontrol.com/stocks/sectors/castings-forgings.html
http://www.moneycontrol.com/stocks/sectors

## Pharma


In [9]:
pharma = stock_data_to_csv('http://www.moneycontrol.com/stocks/sectors/pharmaceuticals.html')
pharma

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Completed


,Stock Name,1 Year Return,1 Year Return Percentile,9 Month Return,9 Month Return Percentile,6 Month Return,6 Month Return Percentile,3 Month Return,3 Month Return Percentile,Price,HQM Score,No. of shares to buy
0,Neuland Lab,278.2,100.0,129.55,98.75,192.95,100.0,61.95,100.0,4110.85,99.6875,30
1,Marksans Pharma,137.11,98.75,137.11,100.0,78.82,95.0,51.83,93.75,118.2,96.875,1057
2,Glenmark,106.0,97.5,93.75,97.5,89.62,98.75,38.13,81.25,801.35,93.75,155
3,Aurobindo Pharm,49.93,86.25,76.41,95.0,82.62,97.5,42.34,86.25,863.15,91.25,144
4,Lupin,56.02,88.75,44.16,88.75,59.51,92.5,40.69,85.0,1076.9,88.75,116
...,...,...,...,...,...,...,...,...,...,...,...,...
75,Ortin Labs,-19.02,10.0,-16.28,7.5,-13.73,2.5,0.25,15.0,19.8,8.75,6313
76,Astec Life,-29.48,5.0,-35.72,2.5,-6.62,11.25,-0.91,10.0,1338.4,7.1875,93
77,Take Solutions,-29.68,3.75,-25.15,5.0,-11.22,6.25,0.0,13.125,18.6,7.03125,6720
78,Mangalam Drugs,-27.01,6.25,-26.06,3.75,-10.48,7.5,-4.68,3.75,105.95,5.3125,1179


## Chemicals

In [10]:
chemicals = stock_data_to_csv('http://www.moneycontrol.com/stocks/sectors/chemicals.html')
chemicals

Completed


,Stock Name,1 Year Return,1 Year Return Percentile,9 Month Return,9 Month Return Percentile,6 Month Return,6 Month Return Percentile,3 Month Return,3 Month Return Percentile,Price,HQM Score,No. of shares to buy
0,Refex Ind,446.69,100.0,202.37,100.0,129.67,98.275862,89.35,98.275862,669.15,99.137931,257
1,Premier Explo,195.32,98.275862,138.19,98.275862,174.45,100.0,141.39,100.0,1041.0,99.137931,165
2,Chembond Chem,165.28,96.551724,96.91,96.551724,59.71,93.103448,42.89,91.37931,433.6,94.396552,397
3,Himadri Special,74.66,89.655172,71.93,94.827586,79.0,96.551724,35.35,86.206897,165.4,91.810345,1042
4,Foseco India,88.44,91.37931,65.15,93.103448,75.0,94.827586,28.77,82.758621,3339.7,90.517241,51
5,Hindcon Chemica,107.7,94.827586,33.1,86.206897,42.73,89.655172,40.53,89.655172,132.1,90.086207,1305
6,Indo Borax,47.5,84.482759,23.73,84.482759,34.82,86.206897,52.1,96.551724,182.75,87.931034,943
7,Linde India,42.79,82.758621,55.17,91.37931,43.05,91.37931,24.64,77.586207,4960.1,85.775862,34
8,GOCL Corp,47.65,86.206897,51.38,89.655172,21.37,79.310345,31.27,84.482759,420.0,84.913793,410
9,Vadivarhe Speci,100.0,93.103448,40.14,87.931034,9.46,56.896552,44.64,94.827586,40.5,83.189655,4257


In [11]:
comp = stock_data_to_csv('http://www.moneycontrol.com/stocks/sectors/computers-software.html')
comp

Completed


,Stock Name,1 Year Return,1 Year Return Percentile,9 Month Return,9 Month Return Percentile,6 Month Return,6 Month Return Percentile,3 Month Return,3 Month Return Percentile,Price,HQM Score,No. of shares to buy
0,Saksoft,190.19,94.594595,175.6,100.0,122.47,100.0,66.94,97.297297,317.35,97.972973,851
1,Newgen Software,145.24,91.891892,165.74,97.297297,109.79,97.297297,59.17,91.891892,909.35,94.594595,297
2,Zensar Tech,117.11,89.189189,129.92,94.594595,100.08,94.594595,62.0,94.594595,502.6,93.243243,537
3,Innovana,201.36,100.0,121.87,91.891892,68.16,86.486486,39.74,75.675676,422.0,88.513514,640
4,Keerti Knowledg,200.49,97.297297,78.81,86.486486,62.32,83.783784,47.54,81.081081,61.6,87.162162,4387
5,BLACK BOX,70.63,83.783784,56.92,81.081081,91.32,91.891892,49.51,86.486486,221.65,85.810811,1219
6,63 Moons Tech,47.82,81.081081,74.78,83.783784,30.94,78.378378,54.51,89.189189,271.7,83.108108,994
7,Cyient,86.75,86.486486,107.75,89.189189,73.17,89.189189,24.94,62.162162,1542.95,81.756757,175
8,Intellect Desig,12.26,51.351351,55.89,78.378378,51.8,81.081081,48.82,83.783784,694.1,73.648649,389
9,COFORGE LTD.,36.58,78.378378,35.16,75.675676,30.64,72.972973,27.71,64.864865,5267.9,72.972973,51


In [160]:
fert = stock_data_to_csv('http://www.moneycontrol.com/stocks/sectors/fertilisers.html')
fert

Completed


,Stock Name,1 Year Return,1 Year Return Percentile,9 Month Return,9 Month Return Percentile,6 Month Return,6 Month Return Percentile,3 Month Return,3 Month Return Percentile,Price,HQM Score,No. of shares to buy
0,Fert and Chem,400.76,100.0,328.26,100.0,61.83,100.0,47.86,100.0,494.0,100.0,1190
1,Madras Fert,79.16,94.117647,63.65,94.117647,22.27,88.235294,15.58,76.470588,81.25,88.235294,7239
2,GSFC,23.19,64.705882,29.82,82.352941,24.05,94.117647,32.94,94.117647,168.9,83.823529,3482
3,NFL,66.24,88.235294,48.85,88.235294,0.57,58.823529,-13.23,17.647059,71.15,63.235294,8267
4,Zuari Agro Chem,15.16,58.823529,-9.34,29.411765,7.65,76.470588,24.91,88.235294,166.0,63.235294,3543
5,SPIC,39.88,76.470588,29.64,76.470588,-5.67,41.176471,6.8,58.823529,69.1,63.235294,8512
6,Rashtriya Chem,40.12,82.352941,22.22,70.588235,-7.44,35.294118,7.25,64.705882,113.85,63.235294,5166
7,Zuari Ind.,7.97,47.058824,-7.75,41.176471,7.37,64.705882,22.06,82.352941,153.0,58.823529,3844
8,Agri-Tech,12.74,52.941176,0.56,52.941176,-2.52,52.941176,4.0,52.941176,98.7,52.941176,5959
9,Coromandel Int,-2.61,35.294118,-2.36,47.058824,7.46,70.588235,1.96,47.058824,944.1,50.0,623


,Stock Name,1 Year Return,1 Year Return Percentile,9 Month Return,9 Month Return Percentile,6 Month Return,6 Month Return Percentile,3 Month Return,3 Month Return Percentile,Price,HQM Score,No. of shares to buy
0,Neuland Lab,128.65,100.0,94.36,98.765432,78.28,100.0,44.44,97.530864,2967.9,99.074074,41
1,Glenmark,80.35,95.061728,79.63,97.530864,67.13,98.765432,43.2,95.061728,702.1,96.604938,175
2,Marksans Pharma,113.75,98.765432,113.09,100.0,56.71,96.296296,32.76,83.950617,103.35,94.753086,1194
3,Refex Ind,533.67,100.0,364.38,100.0,171.49,100.0,150.8,100.0,727.45,100.0,229
4,Premier Explo,164.34,98.333333,83.44,96.666667,99.43,98.333333,92.7,98.333333,792.75,97.916667,210
5,Chembond Chem,142.46,95.0,127.51,98.333333,66.98,96.666667,49.24,91.666667,420.55,95.416667,396
6,Saksoft,236.71,100.0,235.45,100.0,137.19,100.0,101.75,100.0,322.7,100.0,885
7,Zensar Tech,69.0,88.571429,94.82,94.285714,99.98,97.142857,59.1,94.285714,426.55,93.571429,669
8,Newgen Software,86.45,94.285714,90.07,91.428571,65.73,94.285714,46.73,85.714286,670.55,91.428571,426
9,Saksoft,236.71,100.0,235.45,100.0,137.19,100.0,101.75,100.0,322.7,100.0,885
